In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from data_notebook_test import create_data
from test12 import return_sentiment
from test1 import return_sentiment2
from sklearn.neighbors import KNeighborsClassifier  
from scipy.spatial.distance import euclidean
from scipy.stats import mode
from sklearn.metrics import accuracy_score
import nltk
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/himalibhandari/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
win_data = create_data()

In [3]:
win_data.sort_values(['Year','Candidate'], axis = 0, inplace = True)

In [4]:
win_data

,Year,Candidate,Political Party,Election Result,Candidate Spending ($1mil),Final Favorability Rating (%),Voting-Eligible Population Turnout (%)
0,2000,Albert Gore,Democrat,L,120.3,48.4,54.2
1,2000,George W. Bush,Republican,W,186.5,47.9,54.2
3,2004,George W. Bush,Republican,W,355.0,50.7,60.1
2,2004,John Kerry,Democrat,L,332.7,48.3,60.1
5,2008,Barack Obama,Democrat,W,760.4,52.9,61.6
4,2008,John McCain,Republican,L,239.7,45.6,61.6
7,2012,Barack Obama,Democrat,W,737.1,51.1,58.6
6,2012,Mitt Romney,Republican,L,458.7,47.2,58.6
9,2016,Donald Trump,Republican,W,239.0,46.1,60.1
8,2016,Hillary Clinton,Democrat,L,450.6,48.2,60.1


In [5]:
election_data = pd.read_csv('Sentiment_Total.csv', usecols = [1,3,4,5,6,7])
election_data

,Election_Year,Candidate_Name,Affiliation,Positive_Words,Negative_Words,Total_Sentiment
0,2000,George Bush,R,3517,1574,5091
1,2000,Albert Gore,D,3235,1443,4678
2,2000,George Bush,R,3168,1945,5113
3,2000,Albert Gore,D,2710,1785,4495
4,2000,George Bush,R,3488,1316,4804
5,2000,Albert Gore,D,3200,1231,4431
6,2004,George Bush,R,3182,1450,4632
7,2004,John Kerry,D,2967,1322,4289
8,2004,George Bush,R,2927,2017,4944
9,2004,John Kerry,D,2864,1942,4806


In [7]:
def calc_totals(data,year):
    data_year = data[data['Election_Year']==year].reset_index()
    
    #Republican Candidate Totals: Positive words, Negative words, Total Sentiment
    R_pos = 0 
    R_neg = 0 
    R_sent = 0
    
    #Democatic Candidate Totals: Positive words, Negative words, Total Sentiment
    D_pos = 0
    D_neg = 0
    D_sent = 0
    
    #List to be returned
    totals = []
    
    #Calculate the totals for each candidate per election year
    for i in range(len(data_year)):
        if data_year['Affiliation'][i] == 'R':
            R_pos += data_year['Positive_Words'][i]
            R_neg += data_year['Negative_Words'][i]
            R_sent += data_year['Total_Sentiment'][i]
        else:
            D_pos += data_year['Positive_Words'][i]
            D_neg += data_year['Negative_Words'][i]
            D_sent += data_year['Total_Sentiment'][i]
    
    #Creates the list to be returned
    for i in range(len(set(data_year['Affiliation']))):
        if data_year['Affiliation'][i] == 'R':
            totals.append([data_year['Election_Year'][i],data_year['Candidate_Name'][i],0,R_pos,R_neg,R_sent])
        else:
            totals.append([data_year['Election_Year'][i],data_year['Candidate_Name'][i],1,D_pos,D_neg,D_sent])
    
    return totals

In [8]:
x = calc_totals(election_data,2004)

In [9]:
print(x)

[[2004, 'George Bush', 0, 9248, 4627, 13875], [2004, 'John Kerry', 1, 9006, 4430, 13436]]


In [88]:
def clean_data():
    #Grabs DataFrames from test12 and test1 !!! change names !!!
    sentiment_00_08 = return_sentiment2()
    sentiment_12_16 = return_sentiment()
    
    election_data = pd.concat([sentiment_00_08,sentiment_12_16],join='outer')
    #New list to be turned into DataFrame
    data = []
    
    #Creates List of Totals for new DataFrame
    for year in set(election_data["Election_Year"]):
        data += calc_totals(election_data,year)
    
    #Sorts/creates new DataFrame
    data.sort()
    data = pd.DataFrame(data, columns=['Election_Year','Candidate_Name','Affiliation','Positive_Words','Negative_Words','Total_Sentiment'])
    
    return data

In [21]:
totals = clean_data()

In [22]:
totals

,Election_Year,Candidate_Name,Affiliation,Positive_Words,Negative_Words,Total_Sentiment
0,2000,Albert A. Gore,1,9145,4459,13604
1,2000,George W. Bush,0,10173,4835,15008
2,2004,George W. Bush,0,9248,4627,13875
3,2004,John F. Kerry,1,9006,4430,13436
4,2008,Barack H. Obama,1,7100,4526,11626
5,2008,John S. McCain,0,6758,4208,10966
6,2012,Barack Obama,1,11633,5579,17212
7,2012,Mitt Romney,0,13303,6344,19647
8,2016,Donald Trump,0,13928,12017,25945
9,2016,Hillary Clinton,1,11796,10120,21916


In [89]:
def create_vectors():
    #creates dataframe with total of positive, negative and sentiment words
    totals = clean_data()
    
    #Grabs DataFrame from data_notebook.py
    win_data = create_data()
    win_data.sort_values(['Year','Candidate'], axis = 0, inplace = True)
    
    feature_vector = []
    for i in range(len(totals)):
        feature_vector.append([totals['Election_Year'][i],
                               totals['Affiliation'][i],
                               totals['Positive_Words'][i],
                               totals['Negative_Words'][i],
                               totals['Total_Sentiment'][i],
                               win_data['Candidate Spending ($1mil)'][i],
                               win_data['Final Favorability Rating (%)'][i], 
                               win_data['Voting-Eligible Population Turnout (%)'][i]])
        
    target_vector = list(win_data['Election Result'])
    return feature_vector, target_vector

In [90]:
feature_vector,target_vector = create_vectors()

In [91]:
feature_vector

[[2000, 1, 9145, 4459, 13604, 120.3, 48.4, 54.2],
 [2000, 0, 10173, 4835, 15008, 186.5, 47.9, 54.2],
 [2004, 0, 9248, 4627, 13875, 332.7, 48.3, 60.1],
 [2004, 1, 9006, 4430, 13436, 355.0, 50.7, 60.1],
 [2008, 1, 7100, 4526, 11626, 239.7, 45.6, 61.6],
 [2008, 0, 6758, 4208, 10966, 760.4, 52.9, 61.6],
 [2012, 1, 11633, 5579, 17212, 458.7, 47.2, 58.6],
 [2012, 0, 13303, 6344, 19647, 737.1, 51.1, 58.6],
 [2016, 0, 13928, 12017, 25945, 450.6, 48.2, 60.1],
 [2016, 1, 11796, 10120, 21916, 239.0, 46.1, 60.1]]

In [92]:
target_vector

['L', 'W', 'W', 'L', 'W', 'L', 'W', 'L', 'W', 'L']

In [93]:
x_train, x_test, y_train, y_test = train_test_split(feature_vector,target_vector, test_size=.4)
print(y_test)
print(x_test)

['L', 'L', 'W', 'W']
[[2000, 1, 9145, 4459, 13604, 120.3, 48.4, 54.2], [2004, 1, 9006, 4430, 13436, 355.0, 50.7, 60.1], [2008, 1, 7100, 4526, 11626, 239.7, 45.6, 61.6], [2016, 0, 13928, 12017, 25945, 450.6, 48.2, 60.1]]


In [80]:
def train(X,y,sz):
    x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=sz)
    return x_train, x_test, y_train, y_test

In [82]:
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

classifier = KNeighborsClassifier(n_neighbors = 3, weights = 'distance')
classifier.fit(x_train,y_train)

y_pred = classifier.predict(x_test)

print(y_pred)
print(y_test) 

['L' 'W' 'W' 'L']
['W', 'W', 'W', 'L']


In [83]:
a_score = accuracy_score(y_pred, y_test)

In [84]:
a_score

0.75